In [1]:
Prompt: create a Google Doc (1-2 pages) describing the data wrangling steps you took to clean the dataset. 
Include answers to these questions in your submission:

What kind of cleaning steps did you perform?

How did you deal with missing values, if any?

Were there outliers, and how did you handle them?

SyntaxError: invalid syntax (<ipython-input-1-5a271f8858e8>, line 1)

This is a process documentation of the steps I took to build the base data file for my capstone project. I'm endebted to Micah Blake McCurdy and the amazing work he's done at [Natural Stat Trick](https://www.naturalstattrick.com) where all the flat files were initially gerenated.



Each csv represents a different segment pulled from NST. The labeling convention is as follows:

PK = Penalty Kill
PP = Power Play
S = 5 on 5 or Standard Play
OI = On-Ice (raw counts of an action)
Ind = Indivitual 
Counts = Raw occurances of metric
Rates = occurances over time, often per 60 or per time on ice
2015 = or any other number indicates the season, with 2015 being the year the cup was awarded. so 2015 is the 2014-2015 season

So for example, 'Player Season Totals - Natural Stat Trick-2015PKOIRates.csv' has the data for players On Ice rates during penalty kill situations for the 2014-1015 season. 

I did two bits of pre-processing outside of this notebook. 

1) I adding the first column, 'Year' to each csv, where I added the same style of date format as above, so 2015 means the 2015-2016 NHL season starting October 7th, 2015.

2) to reduce confusion I added 'pk' or 'pp' to the column headers, so there would not be overlapping variables. This could also be delt with in the pd.concat using the key variables but I'd already handled when downloading. If attempting to reproduce this effort from scratch with new data, one would be wise to consider that the file exports have many over lapping namespaces (each file will have 'goals' but no denotation of if they are power play goals, 5-on-5, or short handed) which is important to modify for since each of these factors has a different value in the end product.

In [10]:
## import modules
import csv
import glob
import os
import numpy as np
import pandas as pd
from argparse import Namespace 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [2]:
## import the csvs for processing
os.chdir('/Users/michaelwood/Documents/springboard/springboard/Capstone 1/Springboard Capstone Raw Data Sets')
csvs = glob.glob("*.csv")

## create a new dictionary and pipe in the csvs
player_data = {}
for x in csvs:
    player_data[x.replace(" ","").replace(".csv","").replace("NaturalStatTrick","").replace("--","_")]=pd.read_csv(x)

    


Next I'm going to deal with what I call ['The Sebastian Aho Problem'](https://www.youtube.com/watch?v=5dZo-866LP4) which is that for a time, there were two Sebastian Ahos in the league, so names are an unreliable identifier. Intead I'm concatenating on name, year, and team. Name and birthdate would also likely work here, but since birthday is contained only in one of the files name/year/team will work in it's place. (Certainly most large data sets without primary keys will need one generated as name is almost always redundant but this particular iteration lets me slip in a clever YouTube video; go watch it, it's worth it. I'll wait).

After that I'll flatten the data by year, create a variable for age at the start of the season, and create a Fantasy Points Value. 

In [3]:
print(player_data.keys())

dict_keys(['PlayerSeasonTotals_2014SOIRates', 'PlayerSeasonTotals_2014PKOIRates', 'PlayerSeasonTotals_2014PKIndCounts', 'PlayerSeasonTotals_2015PPOIRates', 'PlayerSeasonTotals_2015PKOIRates', 'PlayerSeasonTotals_2018SOIRates', 'Playerbios2017', 'Playerbios2016', 'PlayerSeasonTotals_2015PKIndCounts', 'PlayerSeasonTotals_2014PPOIRates', 'Playerbios2014', 'Playerbios2015', 'PlayerSeasonTotals_2018PKindCounts', 'PlayerSeasonTotals_2014SIndCounts', 'PlayerSeasonTotals_2017PKindCounts', 'PlayerSeasonTotals_2018PKOIRates', 'PlayerSeasonTotals_2018PPOIRates', 'PlayerSeasonTotals_2016PKindCounts', 'PlayerSeasonTotals_2017SIndCounts', 'PlayerSeasonTotals_2017SOIRates', 'PlayerSeasonTotals_2017PPindCounts', 'PlayerSeasonTotals_2016SIndCounts', 'PlayerSeasonTotals_2018PPindCounts', 'PlayerSeasonTotals_2015SIndCounts', 'PlayerSeasonTotals_2015SOIRates', 'PlayerSeasonTotals_2016PPindCounts', 'PlayerSeasonTotals_2017PKOIRates', 'PlayerSeasonTotals_2016SOIRates', 'Playerbios2018', 'Playerbios2019', 'P

In [3]:
## create index value for each dictionary
## player_data[key] = value.set_index(['Year', 'Player','Team'])
    
    
## create single year variables
player_data_2014 = pd.concat([player_data['PlayerSeasonTotals_2014SOIRates'],player_data['PlayerSeasonTotals_2014PKOIRates'],player_data['PlayerSeasonTotals_2014PKIndCounts'],player_data['PlayerSeasonTotals_2014PPOIRates'], player_data['Playerbios2014'],player_data['PlayerSeasonTotals_2014SIndCounts'],player_data['PlayerSeasonTotals_2014PPindCounts']], axis=1)

In [17]:


len(player_data_2014)

785

In [18]:
player_data_2014.Player.nunique()

785

In [24]:
dfs = [player_data['PlayerSeasonTotals_2014SOIRates'],player_data['PlayerSeasonTotals_2014PKOIRates'],player_data['PlayerSeasonTotals_2014PKIndCounts'],player_data['PlayerSeasonTotals_2014PPOIRates'], player_data['Playerbios2014'],player_data['PlayerSeasonTotals_2014SIndCounts'],player_data['PlayerSeasonTotals_2014PPindCounts']]
cols = ['Player', 'Team']
df = pd.concat([d.set_index(cols) for d in dfs], axis=1)

In [26]:
player_data['PlayerSeasonTotals_2014SOIRates'].loc[player_data['PlayerSeasonTotals_2014SOIRates']]

AttributeError: 'DataFrame' object has no attribute 'Player'